# 第8章 数据规整：聚合、合并和重塑

In [2]:
import pandas as pd
import numpy as np

np.random.seed(2018)

## 8.1 层次化索引

层次化索引（hierarchical indexing）是pandas的⼀项重要功
能，它使你能在⼀个轴上拥有多个（两个以上）索引级别。抽象
点说，它使你能以低维度形式处理⾼维度数据。我们先来看⼀个
简单的例⼦：创建⼀个Series，并⽤⼀个由列表或数组组成的列
表作为索引：

In [8]:
data = pd.Series(np.random.randn(9),
                 index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])

可以看到的结果是经过美化的带有MultiIndex索引的Series的格式。
索引之间的“间隔”表示“直接使⽤上⾯的标签”：

In [10]:
data

a  1    0.433564
   2    0.510221
   3   -0.165131
b  1   -1.351779
   3    0.546631
c  1    1.230655
   2    1.076446
d  2   -1.210625
   3   -0.306677
dtype: float64

In [11]:
data.index

MultiIndex(levels=[['a', 'b', 'c', 'd'], [1, 2, 3]],
           codes=[[0, 0, 0, 1, 1, 2, 2, 3, 3], [0, 1, 2, 0, 2, 0, 1, 1, 2]])

对于⼀个层次化索引的对象，可以使⽤所谓的部分索引，使⽤它
选取数据⼦集的操作更简单

In [5]:
data['b']


1   -1.279487
3    0.502277
dtype: float64

In [6]:
data['b':'c']

b  1   -1.279487
   3    0.502277
c  1    0.856029
   2   -0.142790
dtype: float64

In [13]:
data.loc[['b', 'c']]

b  1   -1.351779
   3    0.546631
c  1    1.230655
   2    1.076446
dtype: float64

有时候可以在“内层”进行选取

In [14]:
data.loc[:,2]

a    0.510221
c    1.076446
d   -1.210625
dtype: float64

层次化索引在数据重塑和基于分组的操作（如透视表⽣成）中扮
演着重要的⻆⾊。例如，可以通过unstack⽅法将这段数据重新
安排到⼀个DataFrame中：

In [15]:
data.unstack()

,1,2,3
a,0.433564,0.510221,-0.165131
b,-1.351779,NaN,0.546631
c,1.230655,1.076446,NaN
d,NaN,-1.210625,-0.306677


unstack的逆运算是stack:

In [16]:
data.unstack().stack()

a  1    0.433564
   2    0.510221
   3   -0.165131
b  1   -1.351779
   3    0.546631
c  1    1.230655
   2    1.076446
d  2   -1.210625
   3   -0.306677
dtype: float64

stack和unstack将在本章后面详细。对于一个DataFrame，每条轴都可以有分层索引

In [18]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                     columns=[['Ohio', 'Ohio', 'Colorado'],
                              ['Green', 'Red', 'Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

各层都可以有名字（可以是字符串，也可以是别的Python对
象）。如果指定了名称，它们就会显示在控制台输出中：

In [31]:
frame.index.names = ['key1', 'key2']

In [32]:
frame.columns.names = ['state', 'color']

In [33]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [34]:
type(frame['Ohio'])

pandas.core.frame.DataFrame

可以单独创建MultiIndex然后复用。上面那个DataFrame中的（带有分级名称）列可以这样创建

**MultiIndex怎么用？？？**

### 重排与分级排序

有时，你需要重新调整某条轴上各级别的顺序，或根据指定级别
上的值对数据进⾏排序。 swaplevel接受两个级别编号或名称，
并返回⼀个互换了级别的新对象（但数据不会发⽣变化）：

In [35]:
frame.swaplevel('key1', 'key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

⽽sort_index则根据单个级别中的值对数据进⾏排序。交换级别
时，常常也会⽤到sort_index，这样最终结果就是按照指定顺序
进⾏字⺟排序了

In [45]:
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [39]:
frame.swaplevel(0, 1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

### 根据级别汇总统计

In [46]:
frame.sum(level='key2')

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [47]:
frame.sum(level='color', axis=1)

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

这其实是利⽤了pandas的groupby功能，本书稍后将对其进⾏详
细讲解。

### 使用DataFrame的列进行索引

⼈们经常想要将DataFrame的⼀个或多个列当做⾏索引来⽤，或
者可能希望将⾏索引变成DataFrame的列。以下⾯这个
DataFrame为例：

In [49]:
frame = pd.DataFrame({'a': range(7), 'b': range(7, 0, -1),
                      'c': ['one', 'one', 'one', 'two', 'two',
                            'two', 'two'],
                      'd': [0, 1, 2, 0, 1, 2, 3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


DataFrame的set_index函数会将其⼀个或多个列转换为⾏索
引，并创建⼀个新的DataFrame：

In [51]:
frame2 = frame.set_index(['c', 'd'])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

默认情况下，那些列会从DataFrame中移除，但也可以将其保留
下来

In [53]:
frame.set_index(['c', 'd'], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

reset_index的功能跟set_index刚好相反，层次化索引的级别会
被转移到列⾥⾯：

In [54]:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


## 8.2 合并数据集

pandas对象中的数据可以通过⼀些⽅式进⾏合并：
- pandas.merge可根据⼀个或多个键将不同DataFrame中的⾏
连接起来。 SQL或其他关系型数据库的⽤户对此应该会⽐较
熟悉，因为它实现的就是数据库的join操作。
- pandas.concat可以沿着⼀条轴将多个对象堆叠到⼀起。
实例⽅法combine_first可以将重复数据编接在⼀起，⽤⼀个对
象中的值填充另⼀个对象中的缺失值。

### 数据库风格的DataFrame合并

数据集的合并（merge）或连接（join）运算是通过⼀个或多个
键将⾏链接起来的。这些运算是关系型数据库（基于SQL）的核
⼼。 pandas的merge函数是对数据应⽤这些算法的主要切⼊点

In [56]:

df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})
df2 = pd.DataFrame({'key': ['a', 'b', 'd'],
                    'data2': range(3)})
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [57]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


这是⼀种多对⼀的合并。 df1中的数据有多个被标记为a和b的
⾏，⽽df2中key列的每个值则仅对应⼀⾏。对这些对象调⽤
merge即可得到：

In [58]:
pd.merge(df1, df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


注意，我并没有指明要⽤哪个列进⾏连接。如果没有指定，
merge就会将重叠列的列名当做键。不过，最好明确指定⼀下：

In [59]:
pd.merge(df1, df2, on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [62]:
df3 = pd.DataFrame({'lkey':['b', 'b', 'a', 'c', 'a', 'a', 'b'], 'data1':range(7)})
df4 = pd.DataFrame({'rkey':['a', 'b', 'd'], 'data2':range(3)})

In [66]:
pd.merge(df3, df4, left_on='lkey', right_on='rkey')

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


可能你已经注意到了，结果⾥⾯c和d以及与之相关的数据消失
了。默认情况下， merge做的是“内连接”；结果中的键是交集。
其他⽅式还有"left"、 "right"以及"outer"。外连接求取的是键的并
集，组合了左连接和右连接的效果

In [67]:
pd.merge(df1, df2, how='outer')

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


对how的选项总结：

|选项 | 说明 |
|:----|:----|
|inner|使用两个表都有的键|
|left|使用左表中所有的键|
|right|使用右表中所有的键|
|outer|使用两个表中所有的键|

多对多的合并有些不直观。看下⾯的例⼦：

In [72]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                    'data1': range(6)})
df2 = pd.DataFrame({'key': ['a', 'b', 'a', 'b', 'd'],
                    'data2': range(5)})
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [73]:
df2

,key,data2
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


In [74]:
pd.merge(df1, df2, on='key', how='left')

,key,data1,data2
0,b,0,1.0
1,b,0,3.0
2,b,1,1.0
3,b,1,3.0
4,a,2,0.0
5,a,2,2.0
6,c,3,NaN
7,a,4,0.0
8,a,4,2.0
9,b,5,1.0


多对多连接产⽣的是⾏的笛卡尔积。由于左边的DataFrame有3
个"b"⾏，右边的有2个，所以最终结果中就有6个"b"⾏。连接⽅
式只影响出现在结果中的不同的键的值：

**笛卡尔积** *是集合的一种,假设A和B都是集合，A和B的笛卡尔积用A X B来表示，是所有有序偶（a,b）的集合，其中a属于A，b属于B。 A X B={（a,b)|a属于A且b属于B} ,则AXB所形成的集合就叫笛卡尔集。 我们有时也称A×B是A与B的直积。*

In [77]:
pd.merge(df1, df2, how='inner')

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,b,5,1
5,b,5,3
6,a,2,0
7,a,2,2
8,a,4,0
9,a,4,2


要根据多个键进⾏合并，传⼊⼀个由列名组成的列表即可：

In [78]:
left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'],
                     'key2': ['one', 'two', 'one'],
                     'lval': [1, 2, 3]})
right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                      'key2': ['one', 'one', 'one', 'two'],
                      'rval': [4, 5, 6, 7]})

结果中会出现哪些键组合取决于所选的合并⽅式，你可以这样来
理解：多个键形成⼀系列元组，并将其当做单个连接键（当然，
实际上并不是这么回事）。

*注意：在进行列-列连接时， DataFrame对象中的索引会被丢弃*

In [79]:
pd.merge(left, right, on=['key1', 'key2'], how='outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


对于合并运算需要考虑的最后⼀个问题是对重复列名的处理。虽
然你可以⼿⼯处理列名重叠的问题（查看前⾯介绍的重命名轴标
签），但merge有⼀个更实⽤的suffixes选项，⽤于指定附加到左
右两个DataFrame对象的重叠列名上的字符串

In [82]:
pd.merge(left, right, on='key1')

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [81]:
pd.merge(left, right, on='key1', suffixes=('_left', '_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7
